<a href="https://colab.research.google.com/github/YashsTiwari/ML_DL_revision/blob/main/Coding_MLP(ANN)_using_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt

In [4]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [6]:
#for reproducibility setting seedvalue
torch.manual_seed(42)

In [7]:
df=pd.read_csv('fashion-mnist_train.csv') ##subset of 6k images from FMNIST DS
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
X=df.iloc[:,1:].values
y=df.iloc[:,0].values

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
X_train=X_train/255.0
X_test=X_test/255.0

In [11]:
class CustomDataset(Dataset):
  def __init__(self,features,labels):
    self.features=torch.tensor(features,dtype=torch.float32)
    self.labels=torch.tensor(labels,dtype=torch.long)

  def __len__(self):
    return len(self.features)

  def __getitem__(self, index):
    return self.features[index], self.labels[index]

In [12]:
train_dataset=CustomDataset(X_train,y_train)
test_dataset=CustomDataset(X_test,y_test)

In [13]:
train_loader=DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True)
test_loader=DataLoader(test_dataset,batch_size=32, shuffle=False, pin_memory=True)

In [14]:
class MyNN(nn.Module):
  def __init__(self, num_features):
    super().__init__()
    self.model=nn.Sequential(
        nn.Linear(num_features,128),
        nn.ReLU(),
        nn.Linear(128,64),
        nn.ReLU(),
        nn.Linear(64,10),
    )

  def forward(self, x):
    return self.model(x)

In [15]:
epochs=100
lr=0.1

In [16]:
model=MyNN(X_train.shape[1])
model.to(device)
criterion=nn.CrossEntropyLoss()
optimizer=optim.SGD(model.parameters(), lr=lr)

In [17]:
for epoch in range(epochs):
  total_epoch_loss=0
  for batch_features, batch_labels in train_loader:
    batch_features, batch_labels=batch_features.to(device), batch_labels.to(device)
    output=model(batch_features)
    loss=criterion(output, batch_labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    total_epoch_loss+=loss.item()
  avg_loss=total_epoch_loss/len(train_loader)
  print(f'Epoch [{epoch+1}], Loss: {avg_loss:.4f}')

Epoch [1], Loss: 0.6353
Epoch [2], Loss: 0.4305
Epoch [3], Loss: 0.3861
Epoch [4], Loss: 0.3585
Epoch [5], Loss: 0.3376
Epoch [6], Loss: 0.3228
Epoch [7], Loss: 0.3079
Epoch [8], Loss: 0.2950
Epoch [9], Loss: 0.2855
Epoch [10], Loss: 0.2747
Epoch [11], Loss: 0.2683
Epoch [12], Loss: 0.2581
Epoch [13], Loss: 0.2494
Epoch [14], Loss: 0.2444
Epoch [15], Loss: 0.2386
Epoch [16], Loss: 0.2316
Epoch [17], Loss: 0.2256
Epoch [18], Loss: 0.2203
Epoch [19], Loss: 0.2121
Epoch [20], Loss: 0.2096
Epoch [21], Loss: 0.2062
Epoch [22], Loss: 0.1999
Epoch [23], Loss: 0.1953
Epoch [24], Loss: 0.1931
Epoch [25], Loss: 0.1876
Epoch [26], Loss: 0.1837
Epoch [27], Loss: 0.1802
Epoch [28], Loss: 0.1733
Epoch [29], Loss: 0.1705
Epoch [30], Loss: 0.1704
Epoch [31], Loss: 0.1670
Epoch [32], Loss: 0.1637
Epoch [33], Loss: 0.1623
Epoch [34], Loss: 0.1572
Epoch [35], Loss: 0.1545
Epoch [36], Loss: 0.1546
Epoch [37], Loss: 0.1462
Epoch [38], Loss: 0.1424
Epoch [39], Loss: 0.1414
Epoch [40], Loss: 0.1423
Epoch [41

In [18]:
model.eval()

MyNN(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [19]:
# evaluation code
total = 0
correct = 0

with torch.no_grad():

  for batch_features, batch_labels in test_loader:
    batch_features, batch_labels=batch_features.to(device), batch_labels.to(device)
    outputs = model(batch_features)
    _, predicted = torch.max(outputs, 1)
    total = total + batch_labels.shape[0]
    correct = correct + (predicted == batch_labels).sum().item()

print(correct/total)

0.8898333333333334
